Especialização em Inteligência Artificial<br>
Recuperação de Informação<br>
Atividade 2 - Implementação do Indexador<br>
Prof. Moisés<br>
Aluno: Fernando dos Santos Alves Fernandes

### Indexador
Ainda no contexto definido na implementação do coletor (Atividade 1), foi implementado o indexador, utilizando como abordagem uma lista invertida.
Para a avaliar a força da palavra ou termo, foram considerados os valores de f(K), F(K) e n(K), implementados em sala de aula, em que o K é a chave ou o termo indexado na lista invertida. Para esse trabalho, o IDF também foi calculado para cada um dos tokens. 
O peso f(K), também conhecido como TF (_Term Frequency_, TF(t,d)) é o número de ocorrências do termo t no documento d. F(K) é o total de ocorrências do termo t (ou chave K), considerando todos os documentos em que ele é encontrado. O peso n(K), que também pode ser encontrado na literatura como DF(t), ou _Document Frequency_ do termo t, corresponde ao número de documentos em que a chave K ocorre. O IDF(t) (_Inverse Document Frequency_, do termo t) é o peso do termo que considera o número de documentos coletados (N) e o IDF do termo e pode ser calculado como $log(N/DF(t))$

A implementação completa do indexador pode ser vista a seguir.

In [7]:
import json
import math
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

class Indexador:
    def __init__(self, coletor) -> None:
        self.coletor = coletor
        #self.tokenized_titles = []
        self.stop_words = set(stopwords.words('portuguese'))
        self.inverted_index = {}
        self.F = {}
        
    def inverted_index_generator(self):
        for object in self.coletor.objects_url:
            tokenized_titles = []
            for title in object.titles: # Os h2!
                tokens = [token for token in [token.lower().replace('\u200b', '') for token in nltk.word_tokenize(title.text) if token.lower() not in self.stop_words] if token.isalnum()] # Lista de palavras relevantes!
                tokenized_titles.extend(tokens)

            f = {}
            for token in tokenized_titles:
                if token not in f:
                    f[token] = 1
                    if token not in self.F:
                        self.F[token] = 0
                else:
                    f[token] += 1
            
            # Term Frequency:
            #         / 1 + log(f) if fi,j(k) > 0, 
            #   tf = { 
            #         \ 0, otherwise
            #
            # Inverse Document Frequency:
            #   idf = log N / ni
            #   N = len((self.coletor.objects_url)
            
            for token in tokenized_titles: # f(k), F(k), n(k), idf
                self.inverted_index.setdefault(token, {}).update({object.url: [f[token], self.F[token], 0, 0]}) # Link e peso do token!
                
            for token in self.inverted_index.keys():
                if token in f:
                    self.F[token] += f[token]
                    
    def update_F(self):
        for token in self.inverted_index.keys():
            for object_url in self.inverted_index[token].keys():
                self.inverted_index[token][object_url][1] = self.F[token] # F(k)
                self.inverted_index[token][object_url][2] = len(self.inverted_index[token].keys()) # n(k)
                self.inverted_index[token][object_url][3] = math.log(len(self.coletor.objects_url)/self.inverted_index[token][object_url][2]) # idf(k)
            
    def save_index(self):
        filename = self.coletor.codigo
        with open(f"index-{filename}.json", 'w') as file:
            json.dump(self.inverted_index, file, indent=4)
        print(f'[indexador] index-{filename}.json = {len(self.inverted_index.keys())}')